In [ ]:

import pandas as pd
from pyecharts import options as opts
from pyecharts.charts import Bar, Tab
from pyecharts.globals import ThemeType

# Load your CSV data
data = pd.read_csv('/content/INFO301 DATA.csv')  # Replace with the actual path to your CSV




# Status labels
status_labels = {0: 'Not Enough Datasets', 1: 'Not Checked Yet', 2: 'Plotted'}

# Add the status labels to the data
data['status_label'] = data[data.columns[2]].map(status_labels)

# Filter data for each status group
data_non_zero = data[data[data.columns[1]] > 0]  # Countries with datasets > 0
data_finished = data_non_zero[data_non_zero['status_label'] == 'Plotted']  # Countries that are finished
data_working = data_non_zero[data_non_zero['status_label'] == 'Not Checked Yet']  # Countries still working on
data_not_enough = data[(data[data.columns[1]] > 0) & (data[data.columns[2]] == 0)]  # Countries with status 0 and datasets > 0

# Additional: Countries without datasets at all (status 0 or datasets == 0)
countries_without_datasets = data[data[data.columns[1]] == 0]  # Countries with 0 datasets

# Create the Tab object for different categories
tab = Tab()

# Function to create a bar chart for each category, ranked by the number of datasets
def create_bar_chart(data, title):
    country_data = data.groupby(data.columns[0])[data.columns[1]].sum()  # Sum the number of datasets for each country
    country_data_sorted = country_data.sort_values(ascending=False)  # Rank by number of datasets in descending order
    bar = (
        Bar(init_opts=opts.InitOpts(theme=ThemeType.CHALK))
        .add_xaxis(country_data_sorted.index.tolist())
        .add_yaxis("Number of Datasets", country_data_sorted.values.tolist())
        .set_global_opts(
            title_opts=opts.TitleOpts(title=title),
            xaxis_opts=opts.AxisOpts(name="Country"),
            yaxis_opts=opts.AxisOpts(name="Number of Datasets")
        )
    )
    return bar

# Add bar charts for each category
tab.add(create_bar_chart(data_non_zero, "Countries with Datasets > 0"), "Datasets > 0")
tab.add(create_bar_chart(data_finished, "Finished Countries (Plotted)"), "Finished")
tab.add(create_bar_chart(data_working, "Working on Countries (Not Checked Yet)"), "Working on")

# Countries with status 0 and datasets > 0
tab.add(create_bar_chart(data_not_enough, "Countries Without Enough Data (Status 0 and Datasets > 0)"), "Not Enough Data (Excluding 0 Datasets)")

# New tab: Countries without datasets at all (just a simple list, no bar chart)
countries_without_datasets_list = countries_without_datasets[data.columns[0]].unique().tolist()

# List the countries without datasets in a new tab (no bar chart)
countries_count = len(countries_without_datasets_list)
table = (
    Bar(init_opts=opts.InitOpts(theme=ThemeType.CHALK))
    .add_xaxis(["Countries without Datasets"])
    .add_yaxis("Count", [countries_count])
    .set_global_opts(
        title_opts=opts.TitleOpts(title="Countries Without Datasets At All"),
        xaxis_opts=opts.AxisOpts(name="No Datasets"),
        yaxis_opts=opts.AxisOpts(name="Number of Countries")
    )
)

tab.add(table, "Countries Without Datasets At All")

# Render the tab as an HTML file
tab.render('table.html')

print("Interactive pyecharts tabs generated and saved as 'countries_status_tabs_5.html'")





Interactive pyecharts tabs generated and saved as 'countries_status_tabs_5.html'
